# Elasticsearch query and filter
By default, Elasticsearch sorts matching search results by relevance score,
which measures how well each document matches a query.
The relevance score is a positive floating point number, returned in the _score metadata field of the search API. The higher the _score, the more relevant the document. While each query type can calculate relevance scores differently, 
score calculation also depends on whether the query clause is run in a query or filter context.

**Note**
Here we are going to use same index **'market_data'** which we have ingested in prevous session.

In [1]:
from elasticsearch import Elasticsearch,ElasticsearchException
host = 'http://localhost:9200/'
user = "elastic"
password = "khan123"
elastic_obj = Elasticsearch([host],http_auth=(user, password)) # elastci_object
index_name = 'market_data'
if not elastic_obj.ping():
    print("Elasticsearch server is not running")
else:
    print("Elastic search engine is running........")


Elastic search engine is running........


In [2]:
def fetch_elastic_data(elastic_index,query):
    try:
        data = elastic_obj.search(index=elastic_index, body=query)
    except ElasticsearchException as e:
        print(str(e))
#     print(data)
    hits = data['hits']['hits']
    return hits

In [3]:
import pandas as pd
def show_result(elastic_result):
    list_dict = []
    for row in elastic_result:
        data = row['_source']
        list_dict.append(data)
    
    datafram = pd.DataFrame(list_dict)
    return datafram 
    

In [4]:
query = {
  "query": {
    "match_all": {}
  }
}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.head(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCM53,18.750,Low Fat,0.052146,Health and Hygiene,104.6280,OUT045,2002,Medium,Tier 2,Supermarket Type1,745.6960
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDI19,15.100,Low Fat,0.052339,Meat,242.7512,OUT046,1997,Small,Tier 1,Supermarket Type1,4119.9704
4,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
5,FDH14,17.100,Regular,0.046770,Canned,141.1838,OUT013,1987,High,Tier 3,Supermarket Type1,2247.7408
6,FDP10,13.650,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
7,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Medium,Tier 2,Supermarket Type1,1076.5986
8,DRK12,9.500,LF,0.041878,Soft Drinks,32.9900,OUT035,2004,Small,Tier 2,Supermarket Type1,133.1600
9,FDY07,11.800,Low Fat,0.000000,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266


In [5]:
query = {
    "size":100,
  "query": {
    "match_all": {}
  }
}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.tail(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
90,FDK21,7.905,Low Fat,0.016759,Snack Foods,250.4408,OUT010,1998,High,Tier 3,Grocery Store,500.6816
91,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
92,FDE36,5.260,Regular,0.041764,Baking Goods,161.8868,OUT035,2004,Small,Tier 2,Supermarket Type1,3275.7360
93,FDP28,13.650,Regular,0.134976,Frozen Foods,260.0936,OUT010,1998,Small,Tier 3,Grocery Store,260.9936
94,FDU02,13.350,Low Fat,0.102492,Dairy,230.5352,OUT035,2004,Small,Tier 2,Supermarket Type1,2748.4224
95,FDX34,8.890,Low Fat,0.071637,Snack Foods,121.7098,OUT027,1985,Medium,Tier 3,Supermarket Type3,4097.3332
96,FDA39,6.320,LF,0.000000,Meat,40.2822,OUT035,2004,Small,Tier 2,Supermarket Type1,1139.1838
97,FDD17,7.500,Low Fat,0.032678,Frozen Foods,239.0906,OUT049,1999,Medium,Tier 1,Supermarket Type1,5942.2650
98,FDY24,4.880,Regular,0.133701,Baking Goods,52.9298,OUT049,1999,Medium,Tier 1,Supermarket Type1,1995.4026
99,FDC37,14.600,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876


In [ ]:
# All setup upto prevous session, Let;s go ahead

# Query Contex:
In the query context, a query clause answers the question “How well does this document match this query clause?” Besides deciding whether or not the document matches, the query clause also calculates a relevance score in the _score metadata field
# Filter Contex:
In a filter context, a query clause answers the question “Does this document match this query clause?” The answer is a simple Yes or No — no scores are calculated. Filter context is mostly used for filtering structured data, e.g.

Does this **timestamp** fall into the range 2015 to 2016?
Is the **status** field set to "Active"?

**Example:**
Let's fetch records where 'Outlet_Size' contain 'Small' and Item_Fat_Content contain 'Item_Fat_Content',filter by establish year


In [55]:
query = {
  "query": { 
    "bool": { 
      "must": [
        { "match": { "Outlet_Size":"Small"}},
        { "match": { "Item_Fat_Content": "Low fat" }}
      ],
      "filter": [ 
        { "range": { "Outlet_Establishment_Year": { "gte": "2007" }}}
      ]
    }
  }
}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.tail(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDS52,8.890,low fat,0.005505,Frozen Foods,102.4016,OUT017,2007,Small,Tier 2,Supermarket Type1,2732.4432
1,DRJ25,14.600,Low Fat,0.151419,Soft Drinks,50.3692,OUT017,2007,Small,Tier 2,Supermarket Type1,1034.6532
2,FDL04,19.000,Low Fat,0.112557,Frozen Foods,104.9622,OUT017,2007,Small,Tier 2,Supermarket Type1,1587.9330
3,FDP25,15.200,Low Fat,0.021327,Canned,216.8824,OUT017,2007,Small,Tier 2,Supermarket Type1,2838.9712
4,DRA12,11.600,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,Small,Tier 2,Supermarket Type1,2552.6772
5,FDT25,7.500,Low Fat,0.051038,Canned,121.7072,OUT017,2007,Small,Tier 2,Supermarket Type1,3552.7088
6,FDW11,12.600,Low Fat,0.049058,Breads,62.7194,OUT017,2007,Small,Tier 2,Supermarket Type1,866.8716
7,FDV38,19.250,Low Fat,0.102350,Dairy,52.7956,OUT017,2007,Small,Tier 2,Supermarket Type1,928.1252
8,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080
9,NCX54,9.195,Low Fat,0.048332,Household,105.1622,OUT017,2007,Small,Tier 2,Supermarket Type1,1693.7952


# Term Query:
Returns documents that contain an exact term in a provided field.
You can use the term query to find documents based on a precise value such as a price, a product ID, or a username.
**<br>Avoid using the term query for text fields**
<br>To search text field values, use the match query instead.

**Note:**
The **query** parameter indicates query context.
The **bool** and two **match** clauses are used in query context, which means that they are used to score how well each document matches.
The **filte**r parameter indicates filter context. Its **range** clauses is used in filter context. They will filter out documents which do not match, but they will not affect the score for matching documents.

**Note**<br>
**term** query match exact match like we do for user_name or email address generally.<br>
IF field is **text** type and we are using **term** query then what happen:<br>
Since **text** type is used in analyser therefore, it is broken in indices in lower case, so it will value/given string stored in lower case,as a result it would find in exact term.
Example:
In our case **Item_Identifier** column is text type.IF we search for 'FDS52','FDS52' it would not match exactly



In [44]:
mapping = elastic_obj.indices.get_mapping(index_name)
mapping

{'market_data': {'mappings': {'properties': {'Item_Fat_Content': {'type': 'text'},
    'Item_Identifier': {'type': 'text'},
    'Item_MRP': {'type': 'float'},
    'Item_Outlet_Sales': {'type': 'float'},
    'Item_Type': {'type': 'text'},
    'Item_Visibility': {'type': 'text'},
    'Item_Weight': {'type': 'float'},
    'Outlet_Establishment_Year': {'type': 'integer'},
    'Outlet_Identifier': {'type': 'text'},
    'Outlet_Location_Type': {'type': 'text'},
    'Outlet_Size': {'type': 'text'},
    'Outlet_Type': {'type': 'text'}}}}}

In [56]:
query = {
  "query": {
    "terms": {
      "Item_Identifier": [ "FDS52", "FDW13" ],
      "boost": 1.0
    }
  }
}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.head(10) # Limit result

""


**No result found,even identifiers exist.
Now convert it into small case and try again**

In [57]:
query = {
  "query": {
    "terms": {
      "Item_Identifier": [ "fds52", "fdw13" ],
      "boost": 1.0
    }
  }
}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.head(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDS52,8.89,low fat,0.005505,Frozen Foods,102.4016,OUT017,2007,Small,Tier 2,Supermarket Type1,2732.4432
1,FDS52,8.89,Low Fat,0.009163,Frozen Foods,101.7016,OUT010,1998,Medium,Tier 3,Grocery Store,101.2016
2,FDW13,8.50,Low Fat,0.098438,Canned,51.1324,OUT017,2007,Small,Tier 2,Supermarket Type1,259.6620


We have found three documents, actually 'Item_Identifier' is text type and it is stored in token with small case,
<br> **"term"** query match exact pattern thats why it was not giving result earliear

In [58]:
# Lets create another index 'term_index' change Item_Identifier type text -> keyword, outlet_size  type text-> keyword
# Just create index below mappiing, I have create index named "market_data_term" as we did earlier

# Elastic search Mapping for dataset

data_mapping = {"mappings":
    {
        "properties": {

            "Item_Identifier": {
                "type": "keyword"
            },
            "Item_Weight": {
                "type": "float"
            },

            "Item_Fat_Content": {
                "type": "text"
            },
            "Item_Visibility": {
                "type": "text"
            },

            "Item_Type": {
                "type": "text"
            },
            "Item_MRP": {
                "type": "float"
            },
            "Outlet_Identifier": {
                "type": "text",
            },
            "Outlet_Establishment_Year": {
                "type": "integer"
            },
            "Outlet_Size": {
                "type": "keyword"
            },
            "Outlet_Location_Type": {
                "type": "text"
            },
            "Outlet_Type": {
                "type": "text"
            },
            "Item_Outlet_Sales": {
                "type": "float"
            },
        }
    }
}
    


In [48]:
# Updateding mapping: we can see here "Item_Identifier,Outlet_Size"
mapping = elastic_obj.indices.get_mapping('market_data_term')
mapping

{'market_data_term': {'mappings': {'properties': {'Item_Fat_Content': {'type': 'text'},
    'Item_Identifier': {'type': 'keyword'},
    'Item_MRP': {'type': 'float'},
    'Item_Outlet_Sales': {'type': 'float'},
    'Item_Type': {'type': 'text'},
    'Item_Visibility': {'type': 'text'},
    'Item_Weight': {'type': 'float'},
    'Outlet_Establishment_Year': {'type': 'integer'},
    'Outlet_Identifier': {'type': 'text'},
    'Outlet_Location_Type': {'type': 'text'},
    'Outlet_Size': {'type': 'keyword'},
    'Outlet_Type': {'type': 'text'}}}}}

In [59]:
# Here term query matched Item_Identifier ,this is exact match
query = {
  "query": {
    "terms": {
      "Item_Identifier": [ "FDS52", "FDW13" ],
      "boost": 1.0
    }
  }
}
records = fetch_elastic_data('market_data_term',query)
df_frame = show_result(records)
df_frame.head(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDS52,8.89,low fat,0.005505,Frozen Foods,102.4016,OUT017,2007,Small,Tier 2,Supermarket Type1,2732.4432
1,FDS52,8.89,Low Fat,0.009163,Frozen Foods,101.7016,OUT010,1998,Medium,Tier 3,Grocery Store,101.2016
2,FDW13,8.50,Low Fat,0.098438,Canned,51.1324,OUT017,2007,Small,Tier 2,Supermarket Type1,259.6620


# Boolean Queries:
A query that matches documents matching boolean combinations of other queries. The bool query maps to Lucene BooleanQuery. It is built using one or more boolean clauses, each clause with a typed occurrence. 
The occurrence types are<br>:
**must** : The clause (query) must appear in matching documents and will contribute to the score<br>.
**filter** : The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching<br>.
**should** : The clause (query) should appear in the matching document<br>.
**must_not** : The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.


In [60]:
query = {"query": {"match_all": {}}}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.head(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCM53,18.750,Low Fat,0.052146,Health and Hygiene,104.6280,OUT045,2002,Medium,Tier 2,Supermarket Type1,745.6960
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDI19,15.100,Low Fat,0.052339,Meat,242.7512,OUT046,1997,Small,Tier 1,Supermarket Type1,4119.9704
4,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
5,FDH14,17.100,Regular,0.046770,Canned,141.1838,OUT013,1987,High,Tier 3,Supermarket Type1,2247.7408
6,FDP10,13.650,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
7,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Medium,Tier 2,Supermarket Type1,1076.5986
8,DRK12,9.500,LF,0.041878,Soft Drinks,32.9900,OUT035,2004,Small,Tier 2,Supermarket Type1,133.1600
9,FDY07,11.800,Low Fat,0.000000,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266


In [86]:
query = {
  "query": { 
    "bool": { 
      "must": 
        { "match": { "Item_Fat_Content":"regular"}},
      "filter": [ 
        { "range": { "Outlet_Establishment_Year": { "gte": "2007" }}}
      ]
    }
  }
}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.tail(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
1,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
2,DRZ11,8.850,Regular,0.113124,Soft Drinks,122.5388,OUT018,2009,Medium,Tier 3,Supermarket Type2,1609.9044
3,FDI04,13.650,Regular,0.073209,Frozen Foods,197.6426,OUT018,2009,Medium,Tier 3,Supermarket Type2,2768.3964
4,FDL50,12.150,Regular,0.042485,Canned,125.0046,OUT018,2009,Medium,Tier 3,Supermarket Type2,1743.0644
5,FDZ16,16.850,Regular,0.160760,Frozen Foods,192.4478,OUT017,2007,Medium,Tier 2,Supermarket Type1,4843.6950
6,FDC60,5.425,Regular,0.115120,Baking Goods,88.3514,OUT017,2007,Medium,Tier 2,Supermarket Type1,1416.8224
7,FDG28,9.285,Regular,0.049559,Frozen Foods,246.4144,OUT017,2007,Small,Tier 2,Supermarket Type1,1225.0720
8,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Medium,Tier 2,Supermarket Type1,4710.5350
9,FDL58,5.780,Regular,0.000000,Snack Foods,263.7568,OUT017,2007,Small,Tier 2,Supermarket Type1,2636.5680


In [92]:
query = {
  "query": { 
    "bool": { 
      "must": 
        { "match": { "Item_Fat_Content":"regular"}},
      "filter": [ 
        { "match": { "Outlet_Size": "Small"}},
        { "range": { "Outlet_Establishment_Year": { "gt": "1997" }}}
      ]
    }
  }
}
records = fetch_elastic_data(index_name,query)
df_frame = show_result(records)
df_frame.tail(10) # Limit result

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDZ03,13.650,Regular,0.078946,Dairy,186.0240,OUT045,2002,Small,Tier 2,Supermarket Type1,1118.5440
1,FDY38,13.600,Regular,0.119418,Dairy,231.0300,OUT045,2002,Small,Tier 2,Supermarket Type1,5359.6900
2,FDB56,8.750,Regular,0.074613,Fruits and Vegetables,187.4556,OUT035,2004,Small,Tier 2,Supermarket Type1,3755.1120
3,FDE36,5.260,Regular,0.041764,Baking Goods,161.8868,OUT035,2004,Small,Tier 2,Supermarket Type1,3275.7360
4,FDP28,13.650,Regular,0.134976,Frozen Foods,260.0936,OUT010,1998,Small,Tier 3,Grocery Store,260.9936
5,FDV10,7.645,Regular,0.066693,Snack Foods,42.3112,OUT035,2004,Small,Tier 2,Supermarket Type1,1065.2800
6,FDG28,9.285,Regular,0.049559,Frozen Foods,246.4144,OUT017,2007,Small,Tier 2,Supermarket Type1,1225.0720
7,FDF24,15.500,Regular,0.042465,Baking Goods,81.5934,OUT010,1998,Small,Tier 3,Grocery Store,327.5736
8,FDL58,5.780,Regular,0.000000,Snack Foods,263.7568,OUT017,2007,Small,Tier 2,Supermarket Type1,2636.5680
9,FDE10,6.670,Regular,0.150555,Snack Foods,130.0626,OUT010,1998,Small,Tier 3,Grocery Store,131.1626
